# Домашняя работа 1. Задание 2

1. Используйте датасет fetch_20newsgroups из sklearn.datasets

2. Сравните разные bag of words подходы (CountVectorizer, TfidfVectorizer) с перебором гиперпараметров, используя логистическую регрессию (гиперпараметры которой нужно подобрать)

3. Реализуйте аналогичное сравнение, но используя алгоритмы на основе бустинга xgboost, lightgbm и catboost

4. Прикрепите ссылку на ваш гитхаб с решением в строке комментариев под домашним заданием


In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from collections import Counter
import numpy as np 

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop_words = stopwords.words('english')

In [2]:
train = fetch_20newsgroups()
test = fetch_20newsgroups(subset="test")

# Сравнение bag-of-words подходов

## CountVectorizer

In [3]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [6]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('clf', LogisticRegression()),
])
params = dict(clf__C=[10, 1, 0.1, 0.01])
grid_search = GridSearchCV(pipeline, params, scoring="accuracy", cv=skf, n_jobs=-1)
grid_search.fit(train["data"], train["target"], )
grid_search.best_score_, grid_search.best_estimator_

/Users/dbevz/opt/anaconda3/envs/r_d/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8642383845235152,
 Pipeline(steps=[('bow', CountVectorizer()), ('clf', LogisticRegression(C=1))]))

In [22]:
train['target_names'][grid_search.predict(['This match was great, they scored 3 goals'])[0]]

'soc.religion.christian'

## TF-IDF

In [21]:
pipeline2 = Pipeline([
    ('bow', TfidfVectorizer()),
    ('clf', LogisticRegression()),
])
params = dict(clf__C=[10, 1, 0.1, 0.01])
grid_search2 = GridSearchCV(pipeline2, params, scoring="accuracy", cv=skf, n_jobs=-1)
grid_search2.fit(train["data"], train["target"], )
grid_search2.best_score_, grid_search.best_estimator_

/Users/dbevz/opt/anaconda3/envs/r_d/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9079011664524309,
 Pipeline(steps=[('bow', CountVectorizer()), ('clf', LogisticRegression(C=1))]))

In [24]:
train['target_names'][grid_search2.predict(['This match was great, they scored 3 goals'])[0]]

'rec.sport.hockey'

##### Score улучшился, и пример стал классифицироваться более правильно

## Алгоритмы бустинга

### XGBoost

+ Without stop words
+ Log regression vs softmax

In [4]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


In [7]:

pipeline3 = Pipeline([
    ('bow', TfidfVectorizer(stop_words=stop_words)),
    ('clf', XGBClassifier(use_label_encoder=False)),
])
# Parameters for xgboost algorith
xgb_params = {
    'clf__n_estimators': [16],
    'clf__max_depth': [8, 12], # Maximum depth of a tree
    'clf__learning_rate': [0.1], 
    'clf__objective': ['reg:logistic', 'multi:softmax'], 
    'clf__eval_metric': ['auc']
    #'num_class':n_classes
}
grid_search3 = GridSearchCV(pipeline3, xgb_params, scoring="accuracy", cv=skf, n_jobs=-1)
grid_search3.fit(train["data"], train["target"], )
grid_search3.best_score_, grid_search3.best_estimator_

(0.7696667250647464,
 Pipeline(steps=[('bow',
                  TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                              'our', 'ours', 'ourselves', 'you',
                                              "you're", "you've", "you'll",
                                              "you'd", 'your', 'yours',
                                              'yourself', 'yourselves', 'he',
                                              'him', 'his', 'himself', 'she',
                                              "she's", 'her', 'hers', 'herself',
                                              'it', "it's", 'its', 'itself', ...])),
                 ('clf',
                  XGBClassifier(base_score=0.5, booster='gbtree',
                                colsample_bylev...
                                interaction_constraints='', learning_rate=0.1,
                                max_delta_step=0, max_depth=12,
                                min_chi

In [8]:
accuracy_score(test['target'], grid_search3.predict(test['data']))

0.7213223579394583

In [10]:
print(classification_report(test['target'], grid_search3.predict(test['data'])))

              precision    recall  f1-score   support

           0       0.72      0.60      0.65       319
           1       0.60      0.65      0.63       389
           2       0.66      0.69      0.67       394
           3       0.59      0.68      0.63       392
           4       0.69      0.73      0.71       385
           5       0.74      0.64      0.69       395
           6       0.74      0.84      0.79       390
           7       0.80      0.73      0.77       396
           8       0.86      0.85      0.86       398
           9       0.82      0.83      0.83       397
          10       0.86      0.85      0.85       399
          11       0.88      0.82      0.85       396
          12       0.41      0.56      0.47       393
          13       0.84      0.68      0.75       396
          14       0.82      0.77      0.79       394
          15       0.78      0.85      0.82       398
          16       0.62      0.78      0.69       364
          17       0.92    

In [9]:
train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

### lightgbm

In [11]:
from lightgbm.sklearn import LGBMClassifier
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier

In [12]:
pipeline4 = Pipeline([
    ('bow', TfidfVectorizer(stop_words=stop_words)),
    ('clf', LGBMClassifier()),
])

lgbm_params = dict(
    #using default values
)

grid_search4 = GridSearchCV(pipeline4, lgbm_params, scoring="accuracy", cv=skf, n_jobs=-1)
grid_search4.fit(train["data"], train["target"], )
grid_search4.best_score_, grid_search4.best_estimator_

(0.8450592787377841,
 Pipeline(steps=[('bow',
                  TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                              'our', 'ours', 'ourselves', 'you',
                                              "you're", "you've", "you'll",
                                              "you'd", 'your', 'yours',
                                              'yourself', 'yourselves', 'he',
                                              'him', 'his', 'himself', 'she',
                                              "she's", 'her', 'hers', 'herself',
                                              'it', "it's", 'its', 'itself', ...])),
                 ('clf', LGBMClassifier())]))

In [27]:
accuracy_score(test['target'], grid_search4.predict(test['data']))

0.7883696229421137

In [28]:
print(classification_report(test['target'], grid_search4.predict(test['data'])))

              precision    recall  f1-score   support

           0       0.79      0.68      0.73       319
           1       0.64      0.74      0.69       389
           2       0.74      0.75      0.75       394
           3       0.61      0.72      0.66       392
           4       0.77      0.80      0.78       385
           5       0.83      0.70      0.76       395
           6       0.83      0.89      0.86       390
           7       0.83      0.82      0.82       396
           8       0.91      0.87      0.89       398
           9       0.89      0.89      0.89       397
          10       0.97      0.91      0.94       399
          11       0.93      0.87      0.90       396
          12       0.56      0.70      0.62       393
          13       0.82      0.78      0.80       396
          14       0.88      0.88      0.88       394
          15       0.89      0.91      0.90       398
          16       0.67      0.85      0.75       364
          17       0.97    

### CatBoost

In [7]:
from catboost import CatBoostClassifier
# https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html

In [8]:
pipeline5 = Pipeline([
    ('bow', TfidfVectorizer(stop_words=stop_words)),
    ('clf', CatBoostClassifier()),
])

cat_params = dict(
    clf__loss_function=['MultiClass'],
    clf__iterations=[100],
    clf__learning_rate=[0.1],
    clf__l2_leaf_reg=[3.0],
    clf__depth=[10],
    
)

grid_search4 = GridSearchCV(pipeline5, cat_params, scoring="accuracy", cv=skf, n_jobs=-1)
grid_search4.fit(train["data"], train["target"], )
grid_search4.best_score_, grid_search5.best_estimator_

RecursionError: maximum recursion depth exceeded while calling a Python object